In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import log_loss
from sklearn.cross_validation import cross_val_score, train_test_split, StratifiedKFold
from sklearn import preprocessing
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import EnsembleVoteClassifier
import re
import xgboost as xgb
color = sns.color_palette()
%matplotlib inline
from xgboost import XGBClassifier

D:\Drace\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_json('final_train.json')
df_test = pd.read_json('final_test.json')

In [86]:
#original_train = pd.read_json('train.json')
#original_test = pd.read_json('test.json')

In [100]:
#df_train = df_train.drop(['manager_id', 'building_id'], 1)
#df_test = df_test.drop(['manager_id', 'building_id'], 1)

In [92]:
#original_train = original_train[['manager_id', 'building_id', 'listing_id']]
#original_test = original_test[['manager_id', 'building_id', 'listing_id']]

In [91]:
#df_train['listing_id'].nunique()

49338

In [94]:
#df_train = df_train.merge(original_train, how = 'left', left_on = 'listing_id', right_on ='listing_id')

In [95]:
#df_test = df_test.merge(original_test, how = 'left', left_on = 'listing_id', right_on = 'listing_id')

In [101]:
#df_train.to_json('C:\\Users\\Drace\\Documents\\final_train.json')
#df_test.to_json('C:\\Users\\Drace\\Documents\\final_test.json')

In [3]:
drace_df = df_train

In [4]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if df_train[f].dtype=='object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df_train[f].values) + list(df_test[f].values))
            df_train[f] = lbl.transform(list(df_train[f].values))
            df_test[f] = lbl.transform(list(df_test[f].values))

Baseline on my current dataframe

In [30]:
feats_used = ['bathrooms', 'bedrooms', 'latitude', 'longitude',"display_address", "manager_id", "building_id", "street_address",\
              'n_log_price', 'price_vs_median_72', 'num_photos', 'num_features', 'num_description_words',\
             'n_num_keyfeat_score','amount_of_caps', 'distance_from_midtown', 'allow_pets','Price_P_Room',\
              'nofee', 'has_phone',\
                'dist_to_nearest_college', 'weekday_created', 'large_space','created', 'created_hour']
x = drace_df[feats_used]
y = drace_df["interest_level"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
rf = RandomForestClassifier(n_estimators=1600, max_depth= 17, min_samples_leaf= 1, oob_score=True)
rf.fit(x_train[feats_used], y_train)
y_pred0 = rf.predict_proba(x_test[feats_used])
log_loss(y_test, y_pred0)

0.56219674648595308

In [31]:
feats_used = ['bathrooms', 'bedrooms', 'latitude', 'longitude',"display_address", "manager_id", "building_id", "street_address",\
              'n_log_price', 'price_vs_median_72', 'num_photos', 'num_features', 'num_description_words',\
             'n_num_keyfeat_score','amount_of_caps', 'distance_from_midtown', 'allow_pets','Price_P_Room',\
              'nofee', 'has_phone',\
                'dist_to_nearest_college', 'weekday_created', 'large_space','created', 'created_hour']
x = drace_df[feats_used]
y = drace_df["interest_level"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
xgb0 = XGBClassifier(n_estimators=2000)
xgb0.fit(x_train,y_train)
y_pred1 = xgb0.predict_proba(x_test[feats_used])
log_loss(y_test, y_pred1)

0.57202916676777904

In [32]:
#some baseline features according to some good rule of thumbs for gbm params

feats_used = ['bathrooms', 'bedrooms', 'latitude', 'longitude',"display_address", "manager_id", "building_id", "street_address",\
              'n_log_price', 'price_vs_median_72', 'num_photos', 'num_features', 'num_description_words',\
             'n_num_keyfeat_score','amount_of_caps', 'distance_from_midtown', 'allow_pets','Price_P_Room',\
              'nofee', 'has_phone',\
                'dist_to_nearest_college', 'weekday_created', 'large_space','created', 'created_hour']
x = drace_df[feats_used]
y = drace_df["interest_level"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
gbm0 = GradientBoostingClassifier(n_estimators=2000, max_features=5, subsample= 0.8)
gbm0.fit(x_train,y_train)
y_pred2 = gbm0.predict_proba(x_test[feats_used])
log_loss(y_test, y_pred2)

0.56687225323876744

In [33]:
ef = ExtraTreesClassifier(n_estimators=2000, max_depth= 25, min_samples_leaf= 5, criterion= 'entropy')
ef.fit(x_train, y_train)
y_pred3 = ef.predict_proba(x_test[feats_used])
log_loss(y_test, y_pred3)

#increase depth lead to less log_loss, entropy, gini seems to work the same

0.62951961225672726

In [17]:
def manager_skill(df):
#new var to create
    new_var = 'manager_id'#'manager_id_encoded'
#response var
    resp_var = 'interest_level'
# Step 1: create manager_skill ranking from training set:
    train_df = pd.read_json("train.json") # upload training scores => test data cannot create a rank skill
    temp = pd.concat([train_df[new_var], pd.get_dummies(train_df[resp_var])], axis = 1).groupby(new_var).mean()
    temp.columns = ['high_frac','low_frac', 'medium_frac']
    temp['count'] = train_df.groupby(new_var).count().iloc[:,1]
    temp['manager_skill'] = temp['high_frac']*2 + temp['medium_frac']
# Step 2: Fill working dataset (e.g. test set) with ranking figures using left-merge to match original_id and temp dfs
    mean_manager_skill = np.mean(temp['manager_skill'])

    skill_df = pd.merge(df[['manager_id']],temp[['manager_skill']],how='left', left_on='manager_id', right_index=True)[['manager_skill']]

    skill_df.fillna(mean_manager_skill, inplace=True)
    df['manager_skill'] = list(skill_df['manager_skill'])
    return df

In [18]:
manager_skill(drace_df)
manager_skill(df_test)

,0_per_72,100_per_72,10_per_72,20_per_72,30_per_72,40_per_72,50_per_72,60_per_72,70_per_72,80_per_72,...,preWar,price,price_vs_median_72,price_vs_median_72_new,quiet_nei,space_desc,street_address,subway,weekday_created,manager_skill
0,2190,3937,2200.5,2350.0,2375.0,2397.0,2495.0,2650.0,2700.0,2840.0,...,0,2950,1.182365,1.182365,0,0,24891,1,5,0.363855
1,2695,7000,2850.0,2875.0,3500.0,3550.0,3650.0,3760.0,3950.0,4000.0,...,1,2850,0.780822,0.780822,0,0,5491,1,4,0.363855
10,2950,12000,4039.5,4525.0,5495.0,5595.0,6000.0,6250.0,6500.0,7700.0,...,0,5800,0.966667,0.966667,0,0,22367,0,4,0.363855
100,1600,40000,1795.4,1850.0,1900.0,1960.0,2012.5,2095.0,2135.0,2200.0,...,0,1700,0.844720,0.844720,0,0,11863,1,6,0.363855
1000,2699,4895,2750.0,2800.0,2881.0,3000.0,3175.0,3498.0,3581.5,3695.0,...,0,3200,1.007874,1.007874,0,0,5764,0,3,0.363855
10000,1675,2200,1710.0,1850.0,1900.0,1947.0,1950.0,1950.0,1957.0,2000.0,...,0,1950,1.000000,1.000000,0,0,4975,1,2,0.363855
10001,2350,6200,2850.0,2880.0,2996.5,3168.0,3225.0,3398.0,3498.5,3790.0,...,0,2995,0.928682,0.928682,1,0,22221,0,2,0.363855
10002,1848,11995,1995.0,2100.0,2125.0,2150.0,2200.0,2200.0,2293.5,2340.0,...,1,2150,0.977273,0.977273,1,0,19660,2,4,0.363855
10003,2575,11250,3000.0,3000.0,3000.0,3200.0,3425.0,3590.0,3735.0,4000.0,...,0,3000,0.875912,0.875912,0,0,15897,1,5,0.363855
10004,2600,5300,2905.0,3192.0,3215.0,3300.0,3847.5,4210.0,4295.0,4300.0,...,1,4495,1.168291,1.168291,0,0,14960,1,3,0.363855


In [26]:
pred_logit = ['bathrooms', 'bedrooms', 'price_vs_median_72', 'n_log_price','n_expensive',
              'n_no_photo', 'num_photos','n_num_keyfeat_score', 'num_description_words', 'has_phone',
              'manager_skill','dist_to_nearest_tube', 'subway', 'weekday_created', 'allow_pets', 'amount_of_caps',
             'laundry', 'preWar','furnished','dishwash','hardwood','fitness','doorman', 'nofee']

## separate the predictors and response:
from sklearn import metrics 
# Training:
x = drace_df[pred_logit]
y = drace_df['interest_level']

from sklearn import linear_model
logit_1 = linear_model.LogisticRegression()
logit_1.fit(x, y)

y_pred = logit_1.predict(x)
y_pred_p = logit_1.predict_proba(x)
print 'Multi Class Log_loss:', metrics.log_loss(y,y_pred_p)

Multi Class Log_loss: 0.688611799154


In [43]:
from sklearn.externals import joblib
joblib.dump(gbm0, 'gbm_model0.pkl')
joblib.dump(rf, 'rf_model0.pkl')
joblib.dump(xgb0, 'xgb0_model2.pkl')

['xgb0_model2.pkl']

In [39]:
x_Test = df_test[feats_used]
rf_preds = rf.predict(x_Test)
gbm_preds = gbm0.predict(x_Test)
ef_preds = ef.predict(x_Test)

In [34]:
x_Test = df_test[feats_used]
eclf = EnsembleVoteClassifier(clfs=[rf, gbm0, xgb0, logit_1, ef], weights=[1, 1, 1, 1, 1], voting='hard')

In [35]:
eclf.fit(x_train, y_train)
y_predE = eclf.predict_proba(x_test[feats_used])
log_loss(y_test, y_predE)

0.58649169005454826

In [14]:
feats_used = ['bathrooms', 'bedrooms', 'latitude', 'longitude',\
              'n_log_price', 'price_vs_median_72', 'num_photos', 'num_features', 'num_description_words', \
             'n_num_keyfeat_score','amount_of_caps', 'distance_from_midtown', 'allow_pets','Price_P_Room',\
              'laundry', 'preWar','furnished','dishwash','hardwood','fitness','doorman','nofee', 'has_phone',\
                'dist_to_nearest_college', 'weekday_created', 'large_space', 'created', 'created_hour']
x_Test = df_test[feats_used]
submissioneclf = pd.DataFrame(eclf.predict_proba(x_Test[feats_used]))
submissioneclf = pd.concat([df_test.reset_index(drop=True), submissioneclf], axis=1)
submissioneclf.rename(columns={0: 'high', 1: 'low', 2: 'medium'}, inplace=True)
submissioneclf = submissioneclf[['listing_id', 'high', 'medium', 'low']]
submissioneclf.head()

,listing_id,high,medium,low
0,7142618,0.028253,0.175651,0.796096
1,7210040,0.069152,0.163080,0.767768
2,6879707,0.018148,0.126565,0.855287
3,6819133,0.473853,0.260284,0.265863
4,6908212,0.025037,0.232058,0.742905


In [15]:
submissioneclf.to_csv('C:\\Users\\Drace\\Documents\\submissionECLF.csv', index = False)

In [16]:
#can we meet default xgboost??
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

feats_used = ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'manager_id', 'building_id', 'display_address', 'street_address',\
              'n_log_price', 'price_vs_median_72', 'num_photos', 'num_features', 'num_description_words',\
             'n_num_keyfeat_score','amount_of_caps', 'distance_from_midtown', 'allow_pets','Price_P_Room',\
              'laundry', 'preWar','furnished','dishwash','hardwood','fitness','doorman','nofee', 'has_phone',\
                'dist_to_nearest_college', 'weekday_created', 'large_space','created', 'created_hour']
train_X = df_train[feats_used]
test_X = df_test[feats_used]

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(df_train['interest_level'].apply(lambda x: target_num_map[x]))
print(train_X.shape, test_X.shape)

preds, model = runXGB(train_X, train_y, test_X, num_rounds=400)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = df_test.listing_id.values
out_df.to_csv("xgb_baseline.csv", index=False)

((49338, 32), (74659, 32))


In [ ]:
gbm2 = GradientBoostingClassifier(n_estimators=80, min_samples_split=241, max_depth=15)
gbm2.fit(x_train,y_train)
y_predgbm2 = gbm2.predict_proba(x_test[feats_used])
log_loss(y_test, y_predgbm2)

In [ ]:
rf2
ef2

In [37]:
feats_used = ['bathrooms', 'bedrooms', 'latitude', 'longitude',"display_address", "manager_id", "building_id", "street_address",\
              'n_log_price', 'price_vs_median_72', 'num_photos', 'num_features', 'num_description_words',\
             'n_num_keyfeat_score','amount_of_caps', 'distance_from_midtown', 'allow_pets','Price_P_Room',\
              'nofee', 'has_phone',\
                'dist_to_nearest_college', 'weekday_created', 'large_space','created', 'created_hour']
x_Test = df_test[feats_used]
submissionRF = rf.predict_proba(x_Test)

In [38]:
submissionRF = pd.DataFrame(submissionRF)

In [39]:
submissionRF = pd.concat([df_test.reset_index(drop=True), submissionRF], axis=1)

In [40]:
submission_rf = submissionRF[['listing_id',0,1, 2]]

In [41]:
submission_rf.rename(columns={0: 'high', 1: 'low', 2: 'medium'}, inplace=True)
submission_rf = submission_rf[['listing_id', 'high', 'medium', 'low']]

D:\Drace\Anaconda2\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [42]:
submission_rf.head()

,listing_id,high,medium,low
0,7142618,0.040053,0.172361,0.787586
1,7210040,0.062883,0.167661,0.769456
2,6879707,0.034459,0.223642,0.741899
3,6819133,0.347606,0.404992,0.247402
4,6908212,0.083659,0.283583,0.632757


In [43]:
submission_rf.to_csv('C:\\Users\\Drace\\Documents\\submissionRF.csv', index = False)

In [177]:
submission_rf.columns

Index([u'listing_id', u'high', u'medium', u'low'], dtype='object')

In [49]:
df_test['price_vs_median_72'] = df_test['price']/df_test['median_72']

In [44]:
feats_used = ['bathrooms', 'bedrooms', 'latitude', 'longitude',"display_address", "manager_id", "building_id", "street_address",\
              'n_log_price', 'price_vs_median_72', 'num_photos', 'num_features', 'num_description_words',\
             'n_num_keyfeat_score','amount_of_caps', 'distance_from_midtown', 'allow_pets','Price_P_Room',\
              'nofee', 'has_phone',\
                'dist_to_nearest_college', 'weekday_created', 'large_space','created', 'created_hour']
x_Test = df_test[feats_used]
submissiongbm0 = pd.DataFrame(gbm0.predict_proba(x_Test[feats_used]))
submissiongbm0 = pd.concat([df_test.reset_index(drop=True), submissiongbm0], axis=1)
submissiongbm0.rename(columns={0: 'high', 1: 'low', 2: 'medium'}, inplace=True)


In [45]:
submissiongbm0 = submissiongbm0[['listing_id', 'high', 'medium', 'low']]
submissiongbm0.head()

,listing_id,high,medium,low
0,7142618,0.036155,0.160576,0.803269
1,7210040,0.176551,0.331781,0.491668
2,6879707,0.056370,0.220025,0.723605
3,6819133,0.564130,0.342202,0.093668
4,6908212,0.013311,0.586646,0.400043


In [46]:
submissiongbm0.to_csv('C:\\Users\\Drace\\Documents\\submissiongbm0.csv', index = False)

In [48]:
feats_used = ['bathrooms', 'bedrooms', 'latitude', 'longitude',"display_address", "manager_id", "building_id", "street_address",\
              'n_log_price', 'price_vs_median_72', 'num_photos', 'num_features', 'num_description_words',\
             'n_num_keyfeat_score','amount_of_caps', 'distance_from_midtown', 'allow_pets','Price_P_Room',\
              'nofee', 'has_phone',\
                'dist_to_nearest_college', 'weekday_created', 'large_space','created', 'created_hour']
x_Test = df_test[feats_used]
submissionxgb0 = pd.DataFrame(xgb0.predict_proba(x_Test[feats_used]))
submissionxgb0 = pd.concat([df_test.reset_index(drop=True), submissionxgb0], axis=1)
submissionxgb0.rename(columns={0: 'high', 1: 'low', 2: 'medium'}, inplace=True)
submissionxgb0 = submissionxgb0[['listing_id', 'high', 'medium', 'low']]
submissionxgb0.head()

,listing_id,high,medium,low
0,7142618,0.015025,0.237241,0.747733
1,7210040,0.020795,0.089373,0.889832
2,6879707,0.025545,0.088076,0.886378
3,6819133,0.396984,0.337985,0.265031
4,6908212,0.009863,0.310999,0.679137


In [49]:
submissionxgb0.to_csv('C:\\Users\\Drace\\Documents\\submissionxgb0.csv', index = False)

In [43]:
from sklearn.grid_search import GridSearchCV
param_test1 = {'n_estimators':range(1000,2001,200)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(max_depth= 10, min_samples_leaf= 1, n_jobs=2, max_features='auto'),\
                        param_grid = param_test1, scoring='log_loss',n_jobs=2,iid=False, cv=5)
gsearch1.fit(drace_df[feats_used],y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=False, n_jobs=2,
       param_grid={'n_estimators': [1000, 1200, 1400, 1600, 1800, 2000]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [44]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: -0.61738, std: 0.00440, params: {'n_estimators': 1000},
  mean: -0.61701, std: 0.00400, params: {'n_estimators': 1200},
  mean: -0.61702, std: 0.00412, params: {'n_estimators': 1400},
  mean: -0.61694, std: 0.00417, params: {'n_estimators': 1600},
  mean: -0.61708, std: 0.00426, params: {'n_estimators': 1800},
  mean: -0.61711, std: 0.00431, params: {'n_estimators': 2000}],
 {'n_estimators': 1600},
 -0.61693791723231717)

In [39]:
from sklearn.grid_search import GridSearchCV
max_depth_range = range(1, 11)
leaf_range = range(1, 11)
param_grid2 = dict(max_depth=max_depth_range, min_samples_leaf=leaf_range)
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators = 80, n_jobs=2, max_features='auto'),\
                        param_grid = param_grid2, scoring='log_loss',n_jobs=2,iid=False, cv=5)
gsearch2.fit(drace_df[feats_used],y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=False, n_jobs=2,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [40]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -0.75644, std: 0.00193, params: {'max_depth': 1, 'min_samples_leaf': 1},
  mean: -0.75492, std: 0.00197, params: {'max_depth': 1, 'min_samples_leaf': 2},
  mean: -0.75538, std: 0.00098, params: {'max_depth': 1, 'min_samples_leaf': 3},
  mean: -0.75566, std: 0.00152, params: {'max_depth': 1, 'min_samples_leaf': 4},
  mean: -0.75543, std: 0.00219, params: {'max_depth': 1, 'min_samples_leaf': 5},
  mean: -0.75618, std: 0.00114, params: {'max_depth': 1, 'min_samples_leaf': 6},
  mean: -0.75610, std: 0.00073, params: {'max_depth': 1, 'min_samples_leaf': 7},
  mean: -0.75555, std: 0.00138, params: {'max_depth': 1, 'min_samples_leaf': 8},
  mean: -0.75531, std: 0.00176, params: {'max_depth': 1, 'min_samples_leaf': 9},
  mean: -0.75506, std: 0.00093, params: {'max_depth': 1, 'min_samples_leaf': 10},
  mean: -0.72852, std: 0.00194, params: {'max_depth': 2, 'min_samples_leaf': 1},
  mean: -0.72968, std: 0.00215, params: {'max_depth': 2, 'min_samples_leaf': 2},
  mean: -0.72738, std: 0.00

In [46]:
from sklearn.grid_search import GridSearchCV
param_test3 = {'max_depth':range(11, 21)}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 1600, min_samples_leaf= 1, n_jobs=2, max_features='auto'),\
                        param_grid = param_test3, scoring='log_loss',n_jobs=2,iid=False, cv=5)
gsearch3.fit(drace_df[feats_used],y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1600, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=False, n_jobs=2,
       param_grid={'max_depth': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [47]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: -0.60886, std: 0.00453, params: {'max_depth': 11},
  mean: -0.60187, std: 0.00469, params: {'max_depth': 12},
  mean: -0.59600, std: 0.00514, params: {'max_depth': 13},
  mean: -0.59135, std: 0.00527, params: {'max_depth': 14},
  mean: -0.58792, std: 0.00567, params: {'max_depth': 15},
  mean: -0.58595, std: 0.00584, params: {'max_depth': 16},
  mean: -0.58496, std: 0.00598, params: {'max_depth': 17},
  mean: -0.58518, std: 0.00623, params: {'max_depth': 18},
  mean: -0.58610, std: 0.00653, params: {'max_depth': 19},
  mean: -0.58797, std: 0.00642, params: {'max_depth': 20}],
 {'max_depth': 17},
 -0.58496246014501274)

In [37]:
adb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=100,learning_rate=1)
adb.fit(x_train, y_train)
y_pred4 = adb.predict_proba(x_test[feats_used])
log_loss(y_test, y_pred4)

#not using this, unsure of why this model is so weak

1.0669556158254909